# 2. Data Preparation

The methods provided here allows you to convert the image data into numpy arrays and to store it in npz-format. It makes it a lot easier for sklear to have the data prepared in numpy arrays, so we can easily load the data and train models.

In [1]:
# import libraries
import PIL as pil
import os as os
import numpy as np
import pandas as pd
import humanize as hm

from skimage import data, io, filters, transform
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
n_classes = 256
image_count = 30607
image_resize_factor = 0.3
image_width = int(382 * image_resize_factor)
image_height = int(345 * image_resize_factor)
image_pixels = image_height * image_width * 3
image_pixels

35226

In [3]:
def class_to_data(path, label):
    """
    Converts a class of images in a given path into a two numpy arrays
    
    Args:
        path (str): The path of the folder which contains the images of one class.
        label (int): The label for the images of the class
    
    Returns:
        X (numpy.ndarray): Array containing a image in each row.
        y (numpy.ndarray): Array containing a label in each row.
    """
    
    
    X = np.empty((0, image_width, image_height, 3), dtype=np.int8)
    y = np.empty(0, dtype=np.int8)

    for image in os.listdir(path):
        if image.endswith('.jpg'):
            try:
                image_path = os.path.join(path, image)
                im = io.imread(image_path)
                
                im = transform.resize(im, (image_width, image_height), mode='reflect')
                X = np.append(X, [im], axis=0)
                y = np.append(y, label)
            except:
                 print('Error: Image is not RGB:', image_path)
                 continue
    return X, y

In [4]:
def classes_to_data(path, n_classes=1):
    """
    Converts classes of images in a given path into a two numpy arrays.
    
    Args:
        path (str): The path of the folder which contains the images of one class.
        n_classes (int): The number of classes to use.
    
    Returns:
        X (numpy.ndarray): Array containing a image in each row.
        y (numpy.ndarray): Array containing a label in each row.
    """
    X = np.empty((0, image_width, image_height, 3), dtype=np.int8)
    y = np.empty(0, dtype=np.int8)
    label = 1
    
    for folder in os.listdir(path):
        class_path = os.path.join(path, folder)
        images, targets = class_to_data(class_path, label)
        X = np.append(X, images, axis=0)
        y = np.append(y, targets)
        label += 1
        if label > n_classes:
            break
    return X, y

Generate a new smaller dataset which takes the first 10 classes. We will resize the data on 30% of their size because of performance. After we found a good model we will take the whole data and train it in the Google Cloud.

In [ ]:
X, y = classes_to_data('../data/data_resized/', n_classes=100)

Error: Image is not RGB: ../data/data_resized/197.speed-boat/197_0037.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0016.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0082.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0036.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0054.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0070.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0073.jpg
Error: Image is not RGB: ../data/data_resized/144.minotaur/144_0023.jpg
Error: Image is not RGB: ../data/data_resized/201.starfish-101/201_0044.jpg
Error: Image is not RGB: ../data/data_resized/201.starfish-101/201_0047.jpg
Error: Image is not RGB: ../data/data_resized/201.starfish-101/201_0071.jpg
Error: Image is not RGB: ../data/data_resized/201.starfish-101/201_0027.jpg
Error: Image is not RGB: ../data/data_resized/137.mars/137_0075.jpg
Error: Image is not RGB: ../data/data_resized/038.

Save the 10 classes with labels as npz-file:

In [ ]:
np.savez('../data/image_data_100', X, y)